In [0]:
from pyspark.sql import functions as F
events = spark.table("workspace.ecommerce.events_delta")
print("Events (Bronze table) loaded successfully!")

In [0]:
label_df = events.groupBy("user_id").agg(
    F.max(
    F.when(F.col("event_type") == "purchase", 1).otherwise(0)
    ).alias("purchased")
)
print("Binary purchase label created successfully!")

In [0]:
label_df.show(5)
label_df.groupBy("purchased").count().show()

In [0]:
# Load Silver feature table
features_df = spark.table("workspace.ecommerce.user_features_silver")

# Join features with label on user_id
training_data = features_df.join(label_df, "user_id")

print("Training dataset created successfully!")

In [0]:
training_data.show(5)
training_data.printSchema()
training_data.count()

In [0]:
from pyspark.sql.functions import col
train_df, test_df = training_data.randomSplit([0.8, 0.2], seed=42)

In [0]:
print("Train Label Distribution:")
train_df.groupBy("purchased").count().show()
print("Test Label Distribution:")
test_df.groupBy("purchased").count().show()